## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.json("/user/sprenner/lookup_table_2_official_mai.json")
df_lookup.createOrReplaceTempView("lookup")

In [5]:
df_mai = spark.read.json("/user/sprenner/training_official_mai_hourly.json")
df_mai = df_mai.withColumn("join_key_mai", concat_ws(':', df_mai["scope"], df_mai["filename"]))
df_mai = df_mai.withColumn("fileScope", df_mai["scope"])
df_mai = df_mai.drop("scope")
df_mai.createOrReplaceTempView("mai")

In [6]:
df_lookup.printSchema()

root
 |-- CHILD_NAME: string (nullable = true)
 |-- CHILD_SCOPE: string (nullable = true)
 |-- DID_TYPE: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- SCOPE: string (nullable = true)
 |-- join_key_lookup: string (nullable = true)



In [7]:
df_mai.count()

589283

In [8]:
df_joined = df_mai.join(df_lookup, df_mai.join_key_mai == df_lookup.join_key_lookup, "left_outer")

In [9]:
df_joined.count()

589283

In [10]:
df_joined.createOrReplaceTempView("joined")

In [11]:
spark.sql("SELECT * FROM joined WHERE join_key_lookup IS NULL").toPandas()
#waren bei non officials 3808 rows
#bei officials 3168 - evtl mehr weil durch das filtern weniger doppelte datasets rausgeschmissen wurden

account  accountHash         clientState  clientStateHash  \
0     bejaeger -479767920    DONE               -878527175         
1     bejaeger -479767920    DONE               -878527175         
2     bejaeger -479767920    DONE               -878527175         
3     bejaeger -479767920    DONE               -878527175         
4     bejaeger -479767920    DONE               -878527175         
5     bejaeger -479767920    DONE               -878527175         
6     bejaeger -479767920    DONE               -878527175         
7     bejaeger -479767920    DONE               -878527175         
8     bejaeger -479767920    DONE               -878527175         
9     bejaeger -479767920    DONE               -878527175         
10    bejaeger -479767920    DONE               -878527175         
11    bejaeger -479767920    DONE               -878527175         
12    bejaeger -479767920    DONE               -878527175         
13    bejaeger -479767920    DONE               -878527175         
14    bejaeger -479767920    DONE               -878527175         
15    bejaeger -479767920    DONE               -878527175         
16    bejaeger -479767920    DONE               -878527175         
17    bejaeger -479767920    DONE               -878527175         
18    bejaeger -479767920    DONE               -878527175         
19    bejaeger -479767920    DONE               -878527175         
20    bejaeger -479767920    DONE               -878527175         
21    bejaeger -479767920    DONE               -878527175         
22    bejaeger -479767920    DONE               -878527175         
23    bejaeger -479767920    DONE               -878527175         
24    bejaeger -479767920    DONE               -878527175         
25    mkbugge  -1662366609   DONE               -878527175         
26    mkbugge  -1662366609   DONE               -878527175         
27    mkbugge  -1662366609   DONE               -878527175         
28    mkbugge  -1662366609   DONE               -878527175         
29    kuchida   829373508    ServiceUnavailable -1394468120        
...       ...         ...                   ...         ...        
9966  bejaeger -479767920    DONE               -878527175         
9967  bejaeger -479767920    ServiceUnavailable -1394468120        
9968  bejaeger -479767920    ServiceUnavailable -1394468120        
9969  bejaeger -479767920    ServiceUnavailable -1394468120        
9970  bejaeger -479767920    ServiceUnavailable -1394468120        
9971  bejaeger -479767920    DONE               -878527175         
9972  bejaeger -479767920    DONE               -878527175         
9973  bejaeger -479767920    DONE               -878527175         
9974  bejaeger -479767920    DONE               -878527175         
9975  bejaeger -479767920    DONE               -878527175         
9976  bejaeger -479767920    DONE               -878527175         
9977  bejaeger -479767920    DONE               -878527175         
9978  chhan    -1099237074   DONE               -878527175         
9979  pshih    -1040503448   DONE               -878527175         
9980  bejaeger -479767920    DONE               -878527175         
9981  bejaeger -479767920    DONE               -878527175         
9982  bejaeger -479767920    DONE               -878527175         
9983  mkbugge  -1662366609   DONE               -878527175         
9984  mkbugge  -1662366609   DONE               -878527175         
9985  mkbugge  -1662366609   DONE               -878527175         
9986  kuchida   829373508    ServiceUnavailable -1394468120        
9987  kuchida   829373508    DONE               -878527175         
9988  kuchida   829373508    ServiceUnavailable -1394468120        
9989  kuchida   829373508    DONE               -878527175         
9990  kuchida   829373508    ServiceUnavailable -1394468120        
9991  kuchida   829373508    DONE               -878527175         
9992  tatsuya   1961187428   DONE               -878527175     

In [12]:
df_joined = df_joined.withColumn('dataset_lookup',when(col("join_key_lookup").isNull(), df_joined["dataset"]).otherwise(df_joined["NAME"]))

## create new dataset column split for updated datasets, retain old ones (where join key is null)

In [13]:
df_joined.describe().toPandas()

summary   account           accountHash         clientState  \
0  count   589283    589283                589283               
1  mean    None      -7.873822054121874E7  None                 
2  stddev  None      1.1303640225554888E9  None                 
3  min     abell     -2123889579           CannotAuthenticate   
4  max     zmarshal  2140382084            SourceNotFound       

         clientStateHash  \
0  589283                  
1  -1.0635692705599144E9   
2  2.7545692038495237E8    
3  -1913538301             
4  634051254               

                                                                                           dataset  \
0  589283                                                                                            
1  None                                                                                              
2  None                                                                                              
3  data12_8TeV.00200805.physics_MinBias.recon.NTUP_MINBIAS.r4768                                     
4  mc16_valid.361034.Pythia8EvtGen_A2MSTW2008LO_minbias_inelastic_low.merge.HITS.e3581_s3028_s2939   

             datasetHash datasetScope      datasetScopeHash dataset_version  \
0  589283                 589283       589283                589283           
1  -1.4774623149099838E7  None         -9.356107326362851E8  None             
2  1.2687544178588228E9   None         6.784957656574693E8   None             
3  -2147401420            data12_8TeV  -2057006107           c1128_m1842      
4  2147445885             mc16_valid   2109319881            x563_m1972       

    dataset_versionHash datatype          datatypeHash eventType  \
0  589283                589283   589283                589283     
1  -3.730470822323909E7  None     2.1895068833955333E8  None       
2  1.1014652354502914E9  None     1.0743326832603383E9  None       
3  -2144917680           AOD      -2070622869           download   
4  2146973161            log      2124512252            download   

  eventTypeHash eventVersion       eventVersionHash  \
0  589283        589283       589283                  
1  5.937808E7    None         -2.0115120046498694E9   
2  0.0           None         3.1656468733263874E8    
3  59378080      1.14.10      -2044360925             
4  59378080      1.16.1       1261702107              

                             filename         filenameHash  \
0  589283                              589283                
1  None                                684033.7623824207     
2  None                                1.239027645346361E9   
3  AOD.05341490._000446.pool.root.1    -2147471303           
4  log.14217468._003414.job.log.tgz.1  2147478625            

        filesize_double                hits                  hostname  \
0  589283                589283              589283                     
1  1.4339739956710258E9  2.9973221694839323  None                       
2  2.1931399445812E9     3.8806741048619724  None                       
3  9576.0                1                   ReCaS-ui-01.cs.infn.it     
4  1.1976018886E10       74                  xenia.nevis.columbia.edu   

           hostnameHash           hour              hourHash  \
0  589283                589283         589283                 
1  -4.071422388835245E7  None           -3414275.108615046     
2  9.756384901185799E8   None           1.2200577022275488E9   
3  -2138306752           2018-05-01 00  -2140578122            
4  2129595856            2018-05-31 23  2146843273             

                                        join_key_1        join_key_1Hash  \
0  589283                                           589283                 
1  None                                             829506.522518043       
2  None                                             1.2384114667895505E9   
3  AOD.05341490._000446.pool.root.12018-05-16 13    -2147482375            
4  log.14217468._003414.job.log.tgz.1

In [14]:
df_joined_dropped = df_joined.drop('dataset','datasetHash','prod_step','prod_stepHash','project','projectHash','run_number','run_numberHash','stream_name','stream_nameHash','datatype','datatypeHash','dataset_version','dataset_versionHash')

In [15]:
split_col = split(df_joined_dropped['dataset_lookup'], "\.")
df_joined_dropped = df_joined_dropped.withColumn('project', split_col.getItem(0))
df_joined_dropped = df_joined_dropped.withColumn('run_number', split_col.getItem(1))
df_joined_dropped = df_joined_dropped.withColumn('stream_name', split_col.getItem(2))
df_joined_dropped = df_joined_dropped.withColumn('prod_step', split_col.getItem(3))
df_joined_dropped = df_joined_dropped.withColumn('datatype', split_col.getItem(4))
df_joined_dropped = df_joined_dropped.withColumn('dataset_version', split_col.getItem(5))

In [16]:
df_joined_dropped.count()

589283

In [17]:
df_joined_dropped = df_joined_dropped.withColumn("dataset", df_joined_dropped['dataset_lookup'])
df_joined_dropped = df_joined_dropped.drop("dataset_lookup")

In [18]:
categoricalFeatures = ['dataset', 'project', 'run_number', 'stream_name', 'prod_step', 'datatype', 'dataset_version']

encoded = df_joined_dropped

for elem in categoricalFeatures:
    encoded = encoded.withColumn(elem + "Hash", hash(encoded[elem]))

In [19]:
encoded = encoded.drop('NAME', 'SCOPE', 'CHILD_NAME', 'CHILD_SCOPE','DID_TYPE','join_key_lookup','join_key_mai','join_key_1', 'join_key_1Hash')

In [20]:
encoded.describe().toPandas()

summary   account           accountHash         clientState  \
0  count   589283    589283                589283               
1  mean    None      -7.873822054121874E7  None                 
2  stddev  None      1.1303640225554886E9  None                 
3  min     abell     -2123889579           CannotAuthenticate   
4  max     zmarshal  2140382084            SourceNotFound       

         clientStateHash datasetScope      datasetScopeHash eventType  \
0  589283                 589283       589283                589283     
1  -1.0635692705599144E9  None         -9.356107326362851E8  None       
2  2.754569203849524E8    None         6.784957656574694E8   None       
3  -1913538301            data12_8TeV  -2057006107           download   
4  634051254              mc16_valid   2109319881            download   

            eventTypeHash eventVersion       eventVersionHash  \
0  589283                  589283       589283                  
1  5.937808E7              None         -2.0115120046498694E9   
2  3.6614799924347706E-10  None         3.165646873326389E8     
3  59378080                1.14.10      -2044360925             
4  59378080                1.16.1       1261702107              

                             filename          filenameHash  \
0  589283                              589283                 
1  None                                684033.7623824207      
2  None                                1.2390276453463614E9   
3  AOD.05341490._000446.pool.root.1    -2147471303            
4  log.14217468._003414.job.log.tgz.1  2147478625             

        filesize_double                hits                  hostname  \
0  589283                589283              589283                     
1  1.4339739956710258E9  2.9973221694839323  None                       
2  2.1931399445811996E9  3.8806741048619715  None                       
3  9576.0                1                   ReCaS-ui-01.cs.infn.it     
4  1.1976018886E10       74                  xenia.nevis.columbia.edu   

           hostnameHash           hour              hourHash protocol  \
0  589283                589283         589283                589283    
1  -4.071422388835245E7  None           -3414275.108615046    None      
2  9.756384901185799E8   None           1.2200577022275488E9  None      
3  -2138306752           2018-05-01 00  -2140578122           davs      
4  2129595856            2018-05-31 23  2146843273            srm       

           protocolHash                 remoteSite         remoteSiteHash  \
0  589283                589283                     589283                  
1  4.4731323734995073E8  None                       -4.1575698659324807E8   
2  1.1036590757188318E9  None                       1.1945524910628097E9    
3  -1607963002           AGLT2_CALIBDISK            -2113133296             
4  2128034789            WUPPERTALPROD_SCRATCHDISK  2141740830              

              scopeHash                           traceId  \
0  589283                589283                             
1  -9.402338591305264E8  None                               
2  6.758805608195461E8   None                               
3  -2057006107           000006b4ce98452580a7493dba70ceb0   
4  2109319881            ffffd5de42064e778bb14610ca50b1a6   

            traceIdHash                    traceIp           traceIpHash  \
0  589283                589283                     589283                 
1  702400.1734803142     None                       9.29222020234132E7     
2  1.2396909189249077E9  None                       1.2243341501847217E9   
3  -2147481854           2001:1458:201:e4::100:28c  -2143850866            
4  2147470133            ::ffff:91.239.204.45       2144071436             

     traceTimeentryUnix         transferStart  \
0  589283                589283                 
1  1.5261951742501495E9  1.5261950767825086E9   
2  716920.4582917626     716914.0264713         
3  1.525125633301146E9   1.52486053181973

In [21]:
#TODO: beim nächsten mal verwenden mal schauen ob hier sortieren noch was ändert
encoded.limit(200).toPandas()

account  accountHash         clientState  clientStateHash  datasetScope  \
0    bchargei -151672475    DONE               -878527175        data15_13TeV   
1    anduncan -383481768    DONE               -878527175        data15_13TeV   
2    yuchengh  1175694101   DONE               -878527175        data15_13TeV   
3    yuchengh  1175694101   OSError            -348090455        data15_13TeV   
4    yuchengh  1175694101   OSError            -348090455        data15_13TeV   
5    yuchengh  1175694101   OSError            -348090455        data15_13TeV   
6    yuchengh  1175694101   OSError            -348090455        data15_13TeV   
7    yuchengh  1175694101   OSError            -348090455        data15_13TeV   
8    yuchengh  1175694101   OSError            -348090455        data15_13TeV   
9    yuchengh  1175694101   DONE               -878527175        data15_13TeV   
10   ylo      -728886115    DONE               -878527175        data15_13TeV   
11   yuchengh  1175694101   DONE               -878527175        data15_13TeV   
12   bejaeger -479767920    DONE               -878527175        data15_13TeV   
13   bejaeger -479767920    DONE               -878527175        data15_13TeV   
14   bejaeger -479767920    DONE               -878527175        data15_13TeV   
15   bejaeger -479767920    DONE               -878527175        data15_13TeV   
16   iruffin  -290269263    DONE               -878527175        data15_13TeV   
17   tuna      593210421    DONE               -878527175        data15_13TeV   
18   tuna      593210421    DONE               -878527175        data15_13TeV   
19   tuna      593210421    DONE               -878527175        data15_13TeV   
20   tuna      593210421    DONE               -878527175        data15_13TeV   
21   tuna      593210421    DONE               -878527175        data15_13TeV   
22   tuna      593210421    DONE               -878527175        data15_13TeV   
23   tuna      593210421    DONE               -878527175        data15_13TeV   
24   tuna      593210421    DONE               -878527175        data15_13TeV   
25   tuna      593210421    DONE               -878527175        data15_13TeV   
26   tuna      593210421    DONE               -878527175        data15_13TeV   
27   tuna      593210421    DONE               -878527175        data15_13TeV   
28   tuna      593210421    DONE               -878527175        data15_13TeV   
29   tuna      593210421    DONE               -878527175        data15_13TeV   
..    ...            ...     ...                      ...                 ...   
170  ysano     1418469109   DONE               -878527175        data16_13TeV   
171  ysano     1418469109   ServiceUnavailable -1394468120       data16_13TeV   
172  ysano     1418469109   DONE               -878527175        data16_13TeV   
173  ysano     1418469109   ServiceUnavailable -1394468120       data16_13TeV   
174  ysano     1418469109   DONE               -878527175        data16_13TeV   
175  ysano     1418469109   ServiceUnavailable -1394468120       data16_13TeV   
176  ysano     1418469109   DONE               -878527175        data16_13TeV   
177  ysano     1418469109   DONE               -878527175        data16_13TeV   
178  ysano     1418469109   DONE               -878527175        data16_13TeV   
179  ysano     1418469109   ServiceUnavailable -1394468120       data16_13TeV   
180  ysano     1418469109   ServiceUnavailable -1394468120       data16_13TeV   
181  ysano     1418469109   DONE               -878527175        data16_13TeV   
182  ysano     1418469109   DONE               -878527175        data16_13TeV   
183  ysano     1418469109   DONE               -878527175        data16_13TeV   
184  ysano     1418469109   DONE               -878527175        data16_13TeV   
185  ysano     1418469109   ServiceUnavailable -1394468120       data16_13TeV   
186  ysano     1418469109   ServiceUnavailable -1394468120       data16_13TeV   
187  ysano     1418469109   ServiceUn

In [22]:
encoded = encoded.sort("traceTimeentryUnix", ascending=True)

In [23]:
encoded.limit(200).toPandas()

account  accountHash         clientState  clientStateHash  datasetScope  \
0    hegormse -2036406190   ServiceUnavailable -1394468120       mc16_13TeV     
1    hegormse -2036406190   DONE               -878527175        mc16_13TeV     
2    hegormse -2036406190   ServiceUnavailable -1394468120       mc16_13TeV     
3    hegormse -2036406190   DONE               -878527175        mc16_13TeV     
4    toyamaza  368790896    DONE               -878527175        mc16_13TeV     
5    ysano     1418469109   DONE               -878527175        data17_13TeV   
6    toyamaza  368790896    DONE               -878527175        mc16_13TeV     
7    toyamaza  368790896    DONE               -878527175        mc16_13TeV     
8    ysano     1418469109   DONE               -878527175        data17_13TeV   
9    ysano     1418469109   DONE               -878527175        data17_13TeV   
10   toyamaza  368790896    DONE               -878527175        mc16_13TeV     
11   cylin    -1265559796   DONE               -878527175        mc16_13TeV     
12   ysano     1418469109   DONE               -878527175        data17_13TeV   
13   ysano     1418469109   DONE               -878527175        data17_13TeV   
14   toyamaza  368790896    DONE               -878527175        mc16_13TeV     
15   ysano     1418469109   DONE               -878527175        data17_13TeV   
16   ysano     1418469109   DONE               -878527175        data17_13TeV   
17   hegormse -2036406190   DONE               -878527175        mc16_13TeV     
18   hegormse -2036406190   DONE               -878527175        mc16_13TeV     
19   hegormse -2036406190   DONE               -878527175        mc16_13TeV     
20   hegormse -2036406190   DONE               -878527175        mc16_13TeV     
21   cylin    -1265559796   DONE               -878527175        mc16_13TeV     
22   toyamaza  368790896    DONE               -878527175        mc16_13TeV     
23   khamano  -1185418172   DONE               -878527175        data17_13TeV   
24   khamano  -1185418172   ServiceUnavailable -1394468120       data17_13TeV   
25   toyamaza  368790896    DONE               -878527175        mc16_13TeV     
26   ysano     1418469109   DONE               -878527175        data17_13TeV   
27   ysano     1418469109   DONE               -878527175        data17_13TeV   
28   ysano     1418469109   DONE               -878527175        data17_13TeV   
29   ysano     1418469109   DONE               -878527175        data17_13TeV   
..     ...            ...    ...                      ...                 ...   
170  ysano     1418469109   DONE               -878527175        data17_13TeV   
171  ysano     1418469109   DONE               -878527175        data17_13TeV   
172  toyamaza  368790896    DONE               -878527175        mc16_13TeV     
173  ysano     1418469109   ServiceUnavailable -1394468120       data17_13TeV   
174  ysano     1418469109   DONE               -878527175        data17_13TeV   
175  ysano     1418469109   DONE               -878527175        data17_13TeV   
176  ysano     1418469109   DONE               -878527175        data17_13TeV   
177  ysano     1418469109   DONE               -878527175        data17_13TeV   
178  ysano     1418469109   DONE               -878527175        data17_13TeV   
179  khamano  -1185418172   DONE               -878527175        data17_13TeV   
180  khamano  -1185418172   ServiceUnavailable -1394468120       data17_13TeV   
181  khamano  -1185418172   ServiceUnavailable -1394468120       data17_13TeV   
182  ysano     1418469109   DONE               -878527175        data17_13TeV   
183  toyamaza  368790896    DONE               -878527175        mc16_13TeV     
184  toyamaza  368790896    DONE               -878527175        mc16_13TeV     
185  ysano     1418469109   DONE               -878527175        data17_13TeV   
186  ysano     1418469109   DONE               -878527175        data17_13TeV   
187  toyamaza  368790896    DONE     

In [25]:
encoded.write.format("json").save("/user/sprenner/training_official_mai_hourly_cleaned_datasets.json")